In [1]:
using DataFrames

In [14]:
cur_table = readtable("../data/tokamak_db.csv")

cur_table[:NG] = map(
    (aa,nn,ii) -> ( pi*aa^2*nn/(1.e14*abs(ii)) ), 
    cur_table[:AMIN], cur_table[:NEL], cur_table[:IP]
)

return

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at /Users/dan/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [3] readtable(::String) at /Users/dan/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] include_string(::Module, ::String, ::String) at /Users/dan/.julia/v0.6/Compat/src/Compat.jl:88
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/dan/.julia/v0.6/IJulia/src/execute_request.jl:180
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/dan/.julia/v0.6/Compat/src/Compat.jl:332
 [8] eventloop(::ZMQ.Socket) at /Users/dan/.julia/v0.6/IJulia/src/eventloop.jl:8
 [

In [15]:
cur_tokamaks = ["all"]

append!(cur_tokamaks, unique(cur_table[:TOK]))

cur_tokamaks = map(cur_tokamak -> lowercase(cur_tokamak), cur_tokamaks)

return

In [16]:
using Interact
using DataStructures

using StringCases
using LaTeXStrings
using Query

using Plots
pgfplots()

Plots.PGFPlotsBackend()

In [22]:
function make_plot(tokamak, title, bins=30)
    if tokamak == "all"
        cur_query = @from cur_row in cur_table begin
            @where 0 < cur_row.NG < 2.5
            @where cur_row.SELDB3 == 1111111111
            @select cur_row.NG
            @collect
        end
    else
        cur_query = @from cur_row in cur_table begin
            @where lowercase(cur_row.TOK) == tokamak
            @where 0 < cur_row.NG < 2.5
            @where cur_row.SELDB3 == 1111111111
            @select cur_row.NG
            @collect
        end
    end
    
    cur_N_G_list = sort(Array(cur_query))
    
    min_N_G = 0.0
    max_N_G = 1.25
    
    cur_boundaries = collect(linspace(min_N_G, max_N_G, bins+1))
    cur_bin_counts = zeros(bins)
    
    for (cur_bin, (cur_min_N_G, cur_max_N_G)) in enumerate(zip(cur_boundaries[1:end-1],cur_boundaries[2:end]))
        cur_count = count(cur_N_G -> cur_min_N_G <= cur_N_G < cur_max_N_G, cur_N_G_list)
        cur_bin_counts[cur_bin] = cur_count
        
        if cur_bin == bins
            cur_count = count(cur_N_G -> cur_N_G == cur_max_N_G, cur_N_G_list)
            cur_bin_counts[cur_bin] += cur_count
        end
    end
    
    cur_bin_counts ./= maximum(cur_bin_counts)
    
    cur_xx = []
    cur_yy = []
    
    for (cur_bin, (cur_min_N_G, cur_max_N_G)) in enumerate(zip(cur_boundaries[1:end-1],cur_boundaries[2:end]))       
        push!(cur_xx, cur_min_N_G)
        push!(cur_yy, 0)
        
        push!(cur_xx, cur_min_N_G)
        push!(cur_yy, cur_bin_counts[cur_bin])
        
        push!(cur_xx, cur_max_N_G)
        push!(cur_yy, cur_bin_counts[cur_bin])
        
        push!(cur_xx, cur_max_N_G)
        push!(cur_yy, 0)
        
        push!(cur_xx, NaN)
        push!(cur_yy, NaN)
    end
    
    cur_plot = plot(cur_xx, cur_yy, fill=true, fillopacity=0.15, opacity=0.5, width=0.5, label="")
    
    work_dict = OrderedDict(
        "Not H-Mode" => (0, 0.25),
        "Greenwaldian" => (0.25, 1),
        "Can Disrupt" => (1, 1.25)
    )
    
    for (cur_label, (tmp_min, tmp_max)) in work_dict
        cur_xx = []
        cur_yy = []
        
        push!(cur_xx, tmp_min)
        push!(cur_yy, 1.25)

        for (cur_bin, (cur_min_N_G, cur_max_N_G)) in enumerate(zip(cur_boundaries[1:end-1],cur_boundaries[2:end]))       
            ( cur_min_N_G >= tmp_max ) && break
            ( tmp_min >= cur_max_N_G ) && continue
                        
            push!(cur_xx, cur_min_N_G)
            push!(cur_yy, cur_bin_counts[cur_bin])

            push!(cur_xx, cur_max_N_G)
            push!(cur_yy, cur_bin_counts[cur_bin])
        end
        
        push!(cur_xx, tmp_max)
        push!(cur_yy, 1.25)
       
        push!(cur_xx, tmp_min)
        push!(cur_yy, 1.25)
                
        plot!(cur_xx, cur_yy, fill=true, opacity=0.1, width=0, label=cur_label)
    end
    
    xlims!(0, 1.25)
    ylims!(0, 1.25)
    
    title!(title)
    xlabel!(latexstring("N_G"))
    ylabel!("Count")
    
    cur_tick_marks = collect(linspace(0,1.25,6))
    cur_tick_labels = String[]
    append!(cur_tick_labels, map(string,cur_tick_marks))
    
    cur_tick_labels[1] = "0"
    xticks!(cur_tick_marks, deepcopy(cur_tick_labels))
    
    cur_tick_labels[1] = ""
    yticks!(cur_tick_marks, deepcopy(cur_tick_labels))
    
    cur_plot
end

make_plot (generic function with 3 methods)

In [23]:
@manipulate for tokamak in reverse(cur_tokamaks), bins=20:5:40
    cur_title = tokamak == "all" ? "ITER Database" : capitalize(tokamak)
    make_plot(tokamak,cur_title, bins)
end

Interact.Options{:ToggleButtons,String}(25: "input-9" = tcv String , "tokamak", "tcv", "tcv", 1, Interact.OptionDict(DataStructures.OrderedDict("tcv"=>"tcv","pdx"=>"pdx","pbxm"=>"pbxm","jt60u"=>"jt60u","jft2m"=>"jft2m","aug"=>"aug","cmod"=>"cmod","compass"=>"compass","d3d"=>"d3d","jet"=>"jet"…), Dict("pbxm"=>"pbxm","textor"=>"textor","d3d"=>"d3d","jft2m"=>"jft2m","t10"=>"t10","jt60u"=>"jt60u","tcv"=>"tcv","cmod"=>"cmod","tdev"=>"tdev","tuman3m"=>"tuman3m"…)), Any[], Any[], true, "horizontal", true)

Interact.Options{:SelectionSlider,Int64}(27: "input-10" = 30 Int64 , "bins", 30, "30", 3, Interact.OptionDict(DataStructures.OrderedDict("20"=>20,"25"=>25,"30"=>30,"35"=>35,"40"=>40), Dict(30=>"30",25=>"25",35=>"35",40=>"40",20=>"20")), Any[], Any[], true, "horizontal", true)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
 
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 Z M 4.46875 -1.734375 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.109375 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 4.828125 -6.421875 L 2.40625 -6.421875 C 1.203125 -6.421875 1.171875 -6.546875 1.140625 -6.734375 L 0.890625 -6.734375 L 0.5625 -4.6875 L 0.8125 -4.6875 C 0.84375 -4.84375 0.921875 -5.46875 1.0625 -5.59375 C 1.125 -5.65625 1.90625 -5.65625 2.03125 -5.65625 L 4.09375 -5.65625 L 2.984375 -4.078125 C 2.078125 -2.734375 1.75 -1.34375 1.75 -0.328125 C 1.75 -0.234375 1.75 0.21875 2.21875 0.21875 C 2.671875 0.21875 2.671875 -0.234375 2.671875 -0.328125 L 2.671875 -0.84375 C 2.671875 -1.390625 2.703125 -1.9375 2.78125 -2.46875 C 2.828125 -2.703125 2.953125 -3.5625 3.40625 -4.171875 L 4.75 -6.078125 C 4.828125 -6.1875 4.828125 -6.203125 4.828125 -6.421875 Z M 4.828125 -6.421875 "/>
 
 
<path style="stroke:none;" d="M 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 L 2.9375 -6.375 

In [24]:
cur_dict = Dict()
cur_dict["Alcator C-Mod"] = "cmod"
cur_dict["DIII-D"] = "d3d"
cur_dict["ASDEX Upgrade"] = "asdex"
cur_dict["Joint European Torus"] = "jet"
cur_dict["Entire ITER Database"] = "all"

return

In [27]:
for (cur_key, cur_value) in cur_dict
    cur_plot = make_plot(cur_value,cur_key)
    savefig(cur_plot, "../images/greenwald_$(cur_value).tex")
end